In [1]:
#import something as SM (SM is the alias for the module), so you can use SM to call the module.

# C++ module, when py function is called, it will call the C++ function. This is often used in finance.
import numpy as np
# "panel data" - lets you work with tabular data - known for pandas dataframes
import pandas as pd
# send HTTP requests using Python (internet request that you can send to API to get data)
import requests
# allows you to write data to an Excel file. 
import xlsxwriter
# basic python library to do math operations
import math
# get stock data from Yahoo Finance
import yfinance as yf
#get current date and time
from datetime import datetime


In [2]:
# pandas read html
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# clean data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()
# this list is not survivorship bias free. This means that some of the companies in the list are not in the S&P 500 anymore.

end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*6)

# stacking the data to make it easier to work with
df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

df

[*********************100%%**********************]  503 of 503 completed
C:\Users\Matthew\AppData\Local\Temp\ipykernel_22516\1904382031.py:16: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  end=end_date).stack()


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2018-07-27 A        62.930195   65.750000   66.769997   65.360001   66.650002   
           AAL      38.849533   39.689999   40.860001   39.509998   39.930000   
           AAPL     45.387672   47.744999   48.797501   47.525002   48.747501   
           ABBV     68.985390   90.559998   92.269997   88.769997   91.339996   
           ABT      58.870434   65.260002   65.800003   64.959999   65.510002   
...                       ...         ...         ...         ...         ...   
2024-07-24 XYL     136.710007  136.710007  140.619995  136.520004  140.020004   
           YUM     125.949997  125.949997  126.830002  124.760002  125.970001   
           ZBH     110.550003  110.550003  111.050003  109.510002  110.320000   
           ZBRA    322.089996  322.089996  330.739990  321.660004  328.399994   
           ZTS     179.660004  179.660004  180.839996  178.389999  179.330002   

Price                  Volume  
Date       Ticker              
2018-07-27 A        2638900.0  
           AAL      8445100.0  
           AAPL    96096000.0  
           ABBV    12905800.0  
           ABT      4299000.0  
...                       ...  
2024-07-24 XYL      1010600.0  
           YUM      1620500.0  
           ZBH       925300.0  
           ZBRA      201900.0  
           ZTS      1309300.0  

[746576 rows x 6 columns]

In [4]:
# create empty dataframe
df_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = df_columns)
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [37]:
# df.index is a multiindex. get_level_values(1) returns the 2nd level of the index, as shown above in the df output. unique() removes duplicates.
tickers = df.index.get_level_values(1).unique()

# Create a list of the stock data. Tickers are separated by spaces
# .tickers is a dictionary where keys are tickers and values are Ticker objects from yfinance
thing = yf.Tickers(' '.join(tickers)).tickers

# Access info for each ticker. 
# If previousClose is not available, set it to None, and
# If sharesOutstanding is not available, set it to 0.
# for symbol... is a list comprehension (way to create a list). Here, it specifies the iteration over the dictionary thing.items().
# you have both symbol and ticker, where symbol is the key and ticker is the value. You then have to access more of the data in the ticker object.
data_list = [
    [
        symbol, 
        ticker.info.get('previousClose', None), 
        ticker.info.get('sharesOutstanding', 0) * ticker.info.get('previousClose', 0), 
        'N/A'
    ]
    for symbol, ticker in thing.items()
]

# Finally, create the DataFrame
final_df = pd.DataFrame(data_list, columns=df_columns)

final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,134.16,3.914265e+10,N/A
1,AAL,10.17,6.672435e+09,N/A
2,AAPL,218.54,3.351114e+12,N/A
3,ABBV,176.21,3.111639e+11,N/A
4,ABT,107.26,1.865927e+11,N/A
...,...,...,...,...
498,KVUE,18.17,3.479210e+10,N/A
499,VLTO,99.30,2.451191e+10,N/A
500,GEV,162.90,4.476524e+10,N/A
501,SOLV,58.09,1.003272e+10,N/A


Since this is a simple project, all we are going to do is split a $ portfolio amount into equal amounts for each stock in the S&P 500, then calculuate how many shares we can get for that amount.

In [31]:
# method of checking if int value is inputted, and will continue looping until correct value type
def get_int(prompt="Enter portfolio size ($): "):
    while True:
        try:
            return int(input(prompt))
        # ValueError is raised when the input is not an integer
        except ValueError:
            print("Invalid input. Please try again.")

portfolio_size = get_int()
print(portfolio_size)

position_size = portfolio_size / len(final_df.index)
for i in range(len(final_df.index)):
    # .loc is [row, column]
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Stock Price'])

final_df

1000000


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,134.16,3.914265e+10,14
1,AAL,10.17,6.672435e+09,195
2,AAPL,218.54,3.351114e+12,9
3,ABBV,176.21,3.111639e+11,11
4,ABT,107.26,1.865927e+11,18
...,...,...,...,...
498,KVUE,18.17,3.479210e+10,109
499,VLTO,99.30,2.451191e+10,20
500,GEV,162.90,4.476524e+10,12
501,SOLV,58.09,1.003272e+10,34


In [36]:
# initialize writer. (filename, engine [can also save other types so must specify])
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Recommended Trades', index = False)

# html hex codes
background_color = '#0a0a23'
font_color = '#ffffff'

# dictonary that specifies format of cells in the excel file
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        #solid border around each cell
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        # pass in number with 0s the way you want it to be formatted
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        # solid border around each cell
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        #solid border around each cell
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

# keys() returns the keys of the dictionary (A, B, C, D)
for column in column_formats.keys():
    # looks at dictionary column_formats, at key A-D, and get 2nd value in the list (the format)
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    # goes through A-D and adds 1 (A1, B1...) to change column titles
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.close()